IMPORT LIBRARIES

In [ ]:
import fiftyone as fo
from utils.custom_csv_multilabel_importer import CSVImageClassificationDatasetImporter
import datetime
import os
from pathlib import Path
import platform

work_directory = os.getcwd()
CSV_LABEL = Path(work_directory, "labels_csv", "multilabel_annotations-df_multi.csv")
CSV_LABEL = Path(work_directory, "labels_csv", "ml_anno.csv")
DATASET_DIR = Path(work_directory, "dataset", "multilabel_complete_dataset")
CSV_EXPORT_DIR = Path(work_directory, "export_csv")
IS_NEW_ARCHI = False

# using now() to get current time
current_time = datetime.datetime.now()

dataset_name = 'multilabel_dataset1' + str(current_time.strftime("%Y%m%d_%H%M%S"))

print('Working more or less hard to get it done!')    

### IMPORT MULTILABEL IMAGES AND ANNOTATIONS ###
csv_importer = CSVImageClassificationDatasetImporter(
    dataset_dir = DATASET_DIR,
    csv_labels = CSV_LABEL,
    isNewArchi=IS_NEW_ARCHI
)

In [ ]:
dataset = fo.Dataset.from_importer(
    csv_importer, 
    name=dataset_name, 
    label_field='ground_truth_multilabel',
    progress=True,
    persistent=True # Persistent saved correction made on labels
)

In [ ]:
# Add year, month and day as primitive
dataset.add_sample_field("year", fo.IntField)
dataset.add_sample_field("month", fo.IntField)
dataset.add_sample_field("day", fo.IntField)

for sample in dataset.iter_samples(progress=True):
    date =  sample.filepath.split("\\")[-1].split("_")[0] if platform.platform() == "Windows" else sample.filepath.split("/")[-1].split("_")
    sample.year = int(date[0:4]) if IS_NEW_ARCHI else int(date[1])
    sample.month = int(date[4:6]) if IS_NEW_ARCHI else int(date[2])
    sample.day = int(date[6:8]) if IS_NEW_ARCHI else int(date[3])
    sample.save()

In [ ]:
# Store labels in default_classes
with open(CSV_LABEL, "r") as file:
    for row in file:
        if IS_NEW_ARCHI:
            dataset.default_classes = row.replace("\n", "").split(',')[1:]
        else:
            dataset.default_classes = row.replace("\n", "").split(',')[2:]
        break

In [ ]:
session = fo.launch_app(dataset, auto=False)

In [ ]:
from utils.custom_csv_multilabel_exporter import CSVImageClassificationDatasetExporter

csv_exporter = CSVImageClassificationDatasetExporter(
    export_dir=CSV_EXPORT_DIR,
    dataset_name=dataset_name,
    default_classes=dataset.default_classes
)

dataset.export(dataset_exporter=csv_exporter)

# Delete all existing dataset

In [2]:
import fiftyone as fo
for dt in fo.list_datasets():
    fo.delete_dataset(dt) 

# Saved the first dataset

In [ ]:
from utils.custom_csv_multilabel_exporter import CSVImageClassificationDatasetExporter
import fiftyone as fo
import datetime

CSV_EXPORT_DIR = Path(work_directory, "export_csv")

dataset = fo.load_dataset(fo.list_datasets()[0])

csv_exporter = CSVImageClassificationDatasetExporter(
    export_dir=CSV_EXPORT_DIR,
    dataset_name=str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")),
    default_classes=dataset.default_classes
)

dataset.export(dataset_exporter=csv_exporter)

In [3]:
import fiftyone as fo
session = fo.launch_app(auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


In [1]:
import fiftyone as fo
print(fo.list_datasets())


['20221021_ARM01_00', '20221021_SYC-aldabraDUBOIS_ASV-01_01', '20221022_SYC-aldabraARM06_ASV-01_01', '20221024_SYC-aldabraARM01_ASV-01_01', '20230908_REU-BOUCAN_ASV-1_01', '20230908_REU-BOUCAN_ASV-1_01_2', '20231019_REU-SAINTLEU_ASV-2_02', '20231108_REU-SAINTLEUPORT_ASV-2_02', '20231208_REU-SAINTLEU_ASV-1_01', '20240405_REU-TESSIER_ASV-1_01', 'test']
